In [1]:
import pandas as pd
import numpy as np

Substitute for own data path

In [2]:
df = pd.read_csv(r'C:\Users\eric7\OneDrive\Desktop\aps360Project\convolutional-neural-nuts\data\set.csv')
print("successfully retrieved csv file")

successfully retrieved csv file


IN CSV files, formate as [path:genus]
Created a dictionary that has keys as genus, and items as image paths

In [10]:
grouped = df.groupby('Genus')['Path'].apply(list).to_dict()

mapping = {key: np.array(value) for key, value in grouped.items()}

Analysis of data
- help decide cnn approach
- all data in a numpy dictionary called mapping

In [11]:
print(f"number of classes: {len(mapping)}")

number of classes: 70


In [13]:
LessThan75 = 0
MoreThan75 = 0
totalCount = 0
for dict_keys in mapping.keys():
    totalCount += len(mapping[dict_keys])
    if len(mapping[dict_keys]) > 75:
        print(f"Class name: {dict_keys}, number of images: {len(mapping[dict_keys])}")
        MoreThan75 += 1
    else:
        LessThan75 += 1
print(f"More than 75: {MoreThan75}")
print(f"Less Than 75: {LessThan75}")
print(f"Total Images {totalCount}")

Class name: Amanita, number of images: 130
Class name: Amauroderma, number of images: 133
Class name: Antrodia, number of images: 78
Class name: Aurantiopileus, number of images: 91
Class name: Auricularia, number of images: 389
Class name: Boletinellus, number of images: 96
Class name: Brunneocorticium, number of images: 108
Class name: Calocera, number of images: 125
Class name: Calvatia, number of images: 99
Class name: Camillea, number of images: 112
Class name: Chlorophyllum, number of images: 79
Class name: Clavaria, number of images: 209
Class name: Cookeina, number of images: 719
Class name: Coprinellus, number of images: 228
Class name: Cordierites, number of images: 80
Class name: Coriolopsis, number of images: 92
Class name: Cotylidia, number of images: 112
Class name: Cyathus, number of images: 218
Class name: Cymatoderma, number of images: 117
Class name: Cyptotrama, number of images: 87
Class name: Dacryopinax, number of images: 181
Class name: Dicephalospora, number of i

In [15]:
#variable to track number of clases
n = len(mapping)
print(n)

70


CNN model with transfer learning

In [16]:
#imports
#need to pip install torch here
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image

In [17]:
class preLearnedFungaV1(nn.Module):
    def __init__(self):
        super(preLearnedFungaV1, self).__init__()
        self.name = "fungaV1"
        self.fc1 = nn.Linear(256*6*6, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, n)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = x.squeeze(1)  
        return x

Create training, validation and testing
- apply the transfer learning here

In [18]:
#need to install torchvision
from torch.utils.data import TensorDataset
from torchvision import transforms
#adjust to 224 dimension (input to alexnet)
transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((224,224))])

In [19]:
#use alexnet for now
import torchvision.models
alexnet = torchvision.models.alexnet(pretrained=True)

c:\Users\eric7\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\eric7\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [21]:
#running this takes a long time, only do it after set.csv is fixed
dict = {}
print(f"Total number of classes {n}")
print("Start Transfer Learning Section: AlexNet feature extract for each class")
iterCount = 1

for key, images in mapping.items():
    print(f"Class {iterCount} out of {n}")
    dict[key] = []
    for img in images:
        newImg = Image.open(img)
        newImg = transform(newImg)
        feat = alexnet.features(newImg)
        dict[key].append(feat)
    iterCount += 1


Total number of classes 70
Start Transfer Learning Section: AlexNet feature extract for each class
Class 1 out of 70
Class 2 out of 70
Class 3 out of 70
Class 4 out of 70
Class 5 out of 70
Class 6 out of 70
Class 7 out of 70
Class 8 out of 70
Class 9 out of 70
Class 10 out of 70
Class 11 out of 70
Class 12 out of 70
Class 13 out of 70
Class 14 out of 70
Class 15 out of 70
Class 16 out of 70
Class 17 out of 70
Class 18 out of 70
Class 19 out of 70
Class 20 out of 70
Class 21 out of 70
Class 22 out of 70
Class 23 out of 70
Class 24 out of 70
Class 25 out of 70
Class 26 out of 70
Class 27 out of 70
Class 28 out of 70
Class 29 out of 70
Class 30 out of 70
Class 31 out of 70
Class 32 out of 70
Class 33 out of 70
Class 34 out of 70
Class 35 out of 70
Class 36 out of 70
Class 37 out of 70
Class 38 out of 70
Class 39 out of 70
Class 40 out of 70
Class 41 out of 70
Class 42 out of 70
Class 43 out of 70
Class 44 out of 70
Class 45 out of 70
Class 46 out of 70
Class 47 out of 70
Class 48 out of 7

In [27]:
#change from str key to numerical key
mappingStrToInt = {idx: value for idx, (key, value) in enumerate(dict.items())}
print(len(mappingStrToInt))

70


In [28]:
#numerical dictionary
numericalDict = {}
for idx, (key, value) in enumerate(dict.items()):
    numericalDict[idx] = value
print(len(numericalDict))

70


In [29]:
# Split data into training, validation, and testing sets
tData, tLabel = [], []
vData, vLabel = [], []
testingData, testingLabel = [], []

print("Running data dictionary")
numIter = 1
#0.75 training, 0.15 valdiation, 0.10 for testing
for key, values in numericalDict.items():
    numTrain = int(len(values) * 0.75)
    numVal = int(len(values) * 0.15)

    tLabel.extend([key] * numTrain)
    vLabel.extend([key] * numVal)
    testingLabel.extend([key] * (len(values) - numTrain - numVal))

    tData.extend(values[:numTrain])
    vData.extend(values[numTrain:numTrain + numVal])
    testingData.extend(values[numTrain + numVal:])

    numIter += 1
print("Complete Running Data Dictionary")
tTensor = torch.stack(tData)
vTensor = torch.stack(vData)
testingTensor = torch.stack(testingData)

tLabelTensor = torch.tensor(tLabel)
vLabelTensor = torch.tensor(vLabel)
testingLabelTensor = torch.tensor(testingLabel)

train_set = TensorDataset(tTensor, tLabelTensor)
validation_set = TensorDataset(vTensor, vLabelTensor)
test_set = TensorDataset(testingTensor, testingLabelTensor)

Running data dictionary
Complete Running Data Dictionary


In [31]:
print(f'tTensor shape: {tTensor.shape}')
print(f'vTensor shape: {vTensor.shape}')
print(f'testingTensor shape: {testingTensor.shape}')

tTensor shape: torch.Size([10325, 256, 6, 6])
vTensor shape: torch.Size([2033, 256, 6, 6])
testingTensor shape: torch.Size([1446, 256, 6, 6])


In [30]:
#testing if train, validation, and test sets work
print(len(train_set))
print(len(validation_set))
print(len(test_set))

10325
2033
1446


Data Loading

In [ ]:
#customized data_loading function with variable batch size
import torch
def dataLoadFunction(BatchSize):
  train_load = torch.utils.data.DataLoader(train_set, batch_size = BatchSize, shuffle=True)
  validation_load = torch.utils.data.DataLoader(validation_set, batch_size = BatchSize)
  testing_load = torch.utils.data.DataLoader(test_set, batch_size = BatchSize)
  return train_load, validation_load, testing_load

Training code

In [ ]:
def getValidationLoss(net, validation_loader, criterion):
  n = len(validation_loader)
  totalLoss = 0
  for i, data in enumerate(validation_loader):
    inputs, labels = data
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    totalLoss += loss.item()
  totalLoss = totalLoss / n
  return totalLoss

#model name
def get_model_name(name, batch_size, lr, epoch):
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name, batch_size, lr, epoch)
    return path

#plotting helper function
import matplotlib.pyplot as plt
#plot image function
#modified from lab 2
def plot_training_curve(path):
    train_loss = np.loadtxt("{}_train_loss.csv".format(path))
    val_loss = np.loadtxt("{}_val_loss.csv".format(path))
    n = len(train_loss)
    #plot model
    plt.title("Train vs Validation Loss")
    plt.plot(range(1,n+1), train_loss, label="Train")
    plt.plot(range(1,n+1), val_loss, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend(loc='best')
    plt.show()

In [ ]:
allClasses = []
#^need to loop through all classes in mapping (later problem)

In [ ]:
import time

In [ ]:
def train_net_TL(net, batchsize=24, learning_rate=0.007, num_epochs=12):

  torch.manual_seed(1)

  target_classes = allClasses #defined in last part

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.8)

  training_loader, validation_loader, _ = dataLoadFunction(batchsize)

  train_loss_graph = np.zeros(num_epochs)
  val_loss_graph = np.zeros(num_epochs)

  n = len(training_loader)
  startTime = time.time()
  for currepoch in range(num_epochs):
    #print("New epoch")
    trainLoss = 0
    #net.train()
    for i, (inputs, labels) in enumerate(training_loader):
      optimizer.zero_grad()

      inputs = torch.tensor(inputs.detach().numpy()).to(inputs.device)
      outputs = net(inputs)

      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      trainLoss += loss.item()

    #print("Onto validation")
    #net.eval()
    train_loss_graph[currepoch] = trainLoss/n
    val_loss_graph[currepoch] = getValidationLoss(net, validation_loader, criterion)

    print(f"Epoch: {currepoch} Training Loss: {train_loss_graph[currepoch]:.2f}, Validation Loss: {val_loss_graph[currepoch]:.2f}, Time: {time.time() - startTime:.2f} seconds")

    model_path = get_model_name(net.name, batchsize, learning_rate, currepoch)
    torch.save(net.state_dict(), model_path)
  print("Total time: ", time.time()-startTime)

  path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(net.name, batchsize, learning_rate, num_epochs)
  np.savetxt("{}_train_loss.csv".format(path), train_loss_graph)
  np.savetxt("{}_val_loss.csv".format(path), val_loss_graph)

Training here

In [ ]:
net3TL = preLearnedFungaV1()
train_net_TL(net3TL, batchsize=36, num_epochs=20)

In [ ]:
#plot curve here
plot_training_curve("model_{0}_bs{1}_lr{2}_epoch{3}".format("Lab3Net", 12, 0.007, 8))

Test model

In [ ]:
#test accuracy function
def getAccuracy(net, loader):
  correct, count = 0, 0

  #no gradient modification
  with torch.no_grad():
    print("Total number of batches: ", len(loader))
    for currbatch, (i, corr) in enumerate(loader):
      count += corr.shape[0]
      print("CurrBatch: ", currbatch+1)

      correct += (torch.max(net(i), 1)[1] == corr).sum().item()

  return correct / count

In [ ]:

bestNet = preLearnedFungaV1()
model_path = get_model_name("CHANGENAME", batch_size=24, lr=0.01, epoch=7)
state = torch.load(model_path)
bestNet.load_state_dict(state)

_,_, testLoad = dataLoadFunction(24)

dataLoad = testLoad

accuracy = getAccuracy(bestNet, dataLoad)
print("Test Classification Accuracy: ", accuracy)